In [101]:
import pandas as pd
import statsmodels.api as sm
from IPython.display import display, HTML

In [75]:
df = pd.read_stata("tab5v1.dta")
print(df.columns.tolist())


['id', 'vouch0', 'sex', 'svy', 'age', 'hsvisit', 'strata1', 'strata2', 'strata3', 'strata4', 'strata5', 'strata6', 'mom_sch', 'dad_sch', 'dad_miss', 'mom_miss', 'math', 'reading', 'writing', 't_site', 'totalpts', 'tsite1', 'tsite2', 'tsite3', 'sex_name', 'bog95smp', 'bog95asd', 'bog97smp', 'bog97asd', 'jam93smp', 'jam93asd', 'test_tak', 'dbogota', 'djamundi', 'd1995', 'd1993', 'd1997', 'phone', 'scyfnsh', 'darea1', 'darea2', 'darea3', 'darea4', 'darea5', 'darea6', 'darea7', 'darea8', 'darea9', 'darea10', 'darea11', 'darea12', 'darea13', 'darea14', 'darea15', 'darea16', 'darea17', 'darea18', 'darea19', 'dmonth1', 'dmonth2', 'dmonth3', 'dmonth4', 'dmonth5', 'dmonth6', 'dmonth7', 'dmonth8', 'dmonth9', 'dmonth10', 'dmonth11', 'dmonth12', 'bog95', 'bog97', 'mom_age', 'mom_mw', 'dad_age', 'dad_mw', 'sex2', 'stratams', 'age2']


In [77]:
print(df['scyfnsh'].describe())
print(df['scyfnsh'].value_counts())


count    283.000000
mean       7.749117
std        0.869473
min        5.000000
25%        8.000000
50%        8.000000
75%        8.000000
max       11.000000
Name: scyfnsh, dtype: float64
scyfnsh
8.0     203
7.0      30
6.0      20
9.0      18
5.0       9
11.0      2
10.0      1
Name: count, dtype: int64


In [128]:
df_subset = df[
    (df['test_tak'] == 1) &
    ((df['bog95smp'] == 1) | (df['bog97smp'] == 1) | (df['jam93smp'] == 1))
]


# TABLE 5

## PART A: All Applications

In [131]:
df_subset = df.rename(columns={'id': 'total_points'})
mask = (
    ((df_subset['bog95smp'] == 1) |
     (df_subset['bog97smp'] == 1) |
     (df_subset['jam93smp'] == 1)) &
    (df_subset['test_tak'] == 1)
)
df_exp = df_subset[mask].copy()

outcomes = {
    'Total points': 'scyfnsh',
    'Math scores': 'math',
    'Reading scores': 'reading',
    'Writing scores': 'writing'
}
covariates = ['tsite1', 'tsite2', 'tsite3']

results = []

for label, outcome_var in outcomes.items():
    row = {'Outcome': label}
    
    # OLS with covariates — goes in Column (2)
    df_model2 = df_exp[[outcome_var, 'vouch0'] + covariates].dropna()
    X2 = sm.add_constant(df_model2[['vouch0'] + covariates])
    y2 = df_model2[outcome_var]
    model2 = sm.OLS(y2, X2).fit()
    row['OLS + Covariates'] = f"{model2.params['vouch0']:.3f}<br>({model2.bse['vouch0']:.3f})"
    row['N'] = df_model2.shape[0]

    # OLS without covariates — goes in Column (1)
    df_model1 = df_exp[[outcome_var, 'vouch0']].dropna()
    X1 = sm.add_constant(df_model1['vouch0'])
    y1 = df_model1[outcome_var]
    model1 = sm.OLS(y1, X1).fit()
    row['OLS'] = f"{model1.params['vouch0']:.3f}<br>({model1.bse['vouch0']:.3f})"

    results.append(row)

results_df = pd.DataFrame(results)
results_df = results_df[['Outcome', 'OLS', 'OLS + Covariates', 'N']]
display(HTML(results_df.to_html(escape=False, index=False)))


Outcome,OLS,OLS + Covariates,N
Total points,0.202(0.103),0.212(0.104),271
Math scores,0.152(0.122),0.180(0.122),270
Reading scores,0.182(0.121),0.206(0.121),271
Writing scores,0.149(0.120),0.150(0.121),271


In [132]:
# NEED TO DO SECOND PART

## PART B: Female Applicants

In [135]:
df_female = df_subset[(df_subset['sex'] == 1)].copy()

female_outcomes = {
    'Total points': 'totalpts',
    'Math scores': 'math',
    'Reading scores': 'reading'
}
covariates = ['tsite1', 'tsite2', 'tsite3']

results_female = []

for label, outcome_var in female_outcomes.items():
    row = {'Outcome': label}
    
    df_model2 = df_female[[outcome_var, 'vouch0'] + covariates].dropna()
    X2 = sm.add_constant(df_model2[['vouch0'] + covariates])
    y2 = df_model2[outcome_var]
    model2 = sm.OLS(y2, X2).fit()
    row['OLS + Covariates'] = f"{model2.params['vouch0']:.3f}<br>({model2.bse['vouch0']:.3f})"
    row['N'] = df_model2.shape[0]

    df_model1 = df_female[[outcome_var, 'vouch0']].dropna()
    X1 = sm.add_constant(df_model1['vouch0'])
    y1 = df_model1[outcome_var]
    model1 = sm.OLS(y1, X1).fit()
    row['OLS'] = f"{model1.params['vouch0']:.3f}<br>({model1.bse['vouch0']:.3f})"

    results_female.append(row)

female_df = pd.DataFrame(results_female)[['Outcome', 'OLS', 'OLS + Covariates', 'N']]
display(HTML(female_df.to_html(escape=False, index=False)))


Outcome,OLS,OLS + Covariates,N
Total points,0.209(0.165),0.240(0.164),148
Math scores,0.146(0.163),0.175(0.163),148
Reading scores,0.224(0.166),0.266(0.166),148


In [136]:
# NEED TO DO SECOND PART

## PART C: Male Applicants

In [137]:
df_male = df_subset[df_subset['sex'] == 0].copy()

male_outcomes = {
    'Total points': 'totalpts',
    'Math scores': 'math',
    'Reading scores': 'reading'
}
covariates = ['tsite1', 'tsite2', 'tsite3']

results_male = []

for label, outcome_var in male_outcomes.items():
    row = {'Outcome': label}

    df_model2 = df_male[[outcome_var, 'vouch0'] + covariates].dropna()
    X2 = sm.add_constant(df_model2[['vouch0'] + covariates])
    y2 = df_model2[outcome_var]
    model2 = sm.OLS(y2, X2).fit()
    row['OLS + Covariates'] = f"{model2.params['vouch0']:.3f}<br>({model2.bse['vouch0']:.3f})"
    row['N'] = df_model2.shape[0]

    df_model1 = df_male[[outcome_var, 'vouch0']].dropna()
    X1 = sm.add_constant(df_model1['vouch0'])
    y1 = df_model1[outcome_var]
    model1 = sm.OLS(y1, X1).fit()
    row['OLS'] = f"{model1.params['vouch0']:.3f}<br>({model1.bse['vouch0']:.3f})"

    results_male.append(row)

male_df = pd.DataFrame(results_male)[['Outcome', 'OLS', 'OLS + Covariates', 'N']]
display(HTML(male_df.to_html(escape=False, index=False)))


Outcome,OLS,OLS + Covariates,N
Total points,0.149(0.176),0.188(0.175),134
Math scores,0.125(0.177),0.172(0.174),134
Reading scores,0.117(0.174),0.136(0.175),135


In [138]:
# NEED GTO DO SECOND PART